## Cumm Inline CUDA Tutorial
This Tutorial will show how to write cuda kernels in a simple way.

In [1]:
from cumm.inliner import NVRTCInlineBuilder
from cumm.common import TensorViewNVRTC
from cumm import tensorview as tv

import torch 
import numpy as np 

# we need to init cuda first
torch.zeros([1]).cuda()

tensor([0.], device='cuda:0')

### Simple Kernel 

To write any inliner-based code, you need to specify dependency first.
cumm provides some default dependency that includes ```tensorview/core```:

```Python
from cumm.common import TensorViewNVRTC
```

In [2]:
# reload_when_code_change: should only be used in GUI apps or jupyter environment
inliner = NVRTCInlineBuilder([TensorViewNVRTC], reload_when_code_change=True)


#### Capture-Based CUDA Code

code in cumm.inliner don't contains inputs, all inputs are captured from local variables in current frames.

When you write ```$some_var``` in inline code, cumm.inliner will locate ```some_var``` in current frame and generate code for it.

In [3]:
some_var = 1

inliner.kernel_1d("unique_name_in_a_file", 1, 0, f"""
tv::printf2($some_var);
""")

We support following python types to be captured:

* tv.Tensor -> raw ptr or tv::TensorView
* torch.Tensor -> raw ptr or tv::TensorView
* int/float/bool -> int64_t/float/bool
* np.ndarray -> tv::array, size MUST smaller than 50
* list/tuple of int/float/bool -> tv::array, will be converted to np.ndarray, size MUST smaller than 50

In [4]:
some_arr = np.eye(4)
some_ten_tv = tv.zeros([2], tv.float32, 0)
some_ten_torch = torch.rand(5).cuda()
# here we use a same name as previous kernel_1d but different code, the cached cuda binary will be invalid,
# so we need to run compile again.
# to prevent compile code in every kernel_1d, disable reload_when_code_change.
inliner.kernel_1d("unique_name_in_a_file", 1, 0, f"""
tv::printf2($some_arr[0][0], $some_arr[1][1], $some_arr[0][2], $some_arr[3][3]);
tv::printf2($some_ten_tv[0], $some_ten_tv[1]);
tv::printf2($some_ten_torch[0], $some_ten_torch[1]);
""")

1


```$some_var``` isn't suitable for complex exprs such as ```self.some_var```, so we support another type of capture: ```$(self.complex_expr.dim(0))```

In [5]:
some_arr = np.eye(4)
inliner.kernel_1d("unique_name_in_a_file", 1, 0, f"""
tv::printf2($(some_arr.shape[0]));
""")

1.000000 1.000000 0.000000 1.000000
0.000000 0.000000
0.137446 0.927519


#### 1D kernels or Raw Kernels

When you use kernel_1d, a variable ```i``` will be reversed as a standard 1d kernel index

In [6]:
some_ten = torch.rand(5000).cuda()
inliner.kernel_1d("unique_name_in_a_file", some_ten.shape[0], 0, f"""
$some_ten[i] = 0;
""")
print(some_ten.mean())

4
tensor(0., device='cuda:0')


We also support standard kernel params: blocks, threads, smem size and stream

In [7]:
some_ten = torch.rand(5000).cuda()
inliner.kernel_raw("unique_name_in_a_file", tv.LaunchParam((1, 1, 1), (1024, 1,1), 0, 0), f"""
for (int i = blockIdx.x * blockDim.x + threadIdx.x; 
        i < $(some_ten.shape[0]); 
        i += blockDim.x * gridDim.x) 
{{
    $some_ten[i] = 0;
}}
""")
print(some_ten.mean())

tensor(0., device='cuda:0')


### Real-World kernel examples

Transform Point Cloud in CUDA

In [8]:
# use INLINER as a global variable to avoid recompile

INLINER = NVRTCInlineBuilder([], reload_when_code_change=True)

def transform_pc(pc: torch.Tensor, tr: np.ndarray):
    out_pc = pc.clone()
    INLINER.kernel_1d("transform_pc", pc.shape[0], 0, f"""
    auto pc_ptr = $pc + i * $(pc.stride(0));
    auto out_pc_ptr = $out_pc + i * $(out_pc.stride(0));
    auto x = pc_ptr[0];
    auto y = pc_ptr[1];
    auto z = pc_ptr[2];
    out_pc_ptr[0] = $tr[0][0] * x + $tr[0][1] * y + $tr[0][2] * z + $tr[0][3];
    out_pc_ptr[1] = $tr[1][0] * x + $tr[1][1] * y + $tr[1][2] * z + $tr[1][3];
    out_pc_ptr[2] = $tr[2][0] * x + $tr[2][1] * y + $tr[2][2] * z + $tr[2][3];
    """)
    return out_pc